In [ ]:
!pip install langchain
!pip install langchain_community


ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [ ]:
!pip install transformers

In [ ]:
!pip install pypdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.3 MB/s eta 0:00:00


In [ ]:
import os
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
def download_pdf(url, save_path="temp.pdf"):
    response = requests.get(url)
    with open(save_path, "wb") as file:
        file.write(response.content)
    return save_path

def load_pdf_from_url(url):
    pdf_path = download_pdf(url)
    loader = PyPDFLoader(pdf_path)  # PyPDFLoader 사용
    documents = loader.load()
    os.remove(pdf_path)
    return documents

In [ ]:
def process_pdf_and_create_vectorstore(url, vectorstore_path):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")        # Embedding Model (수정 가능)
    if os.path.exists(vectorstore_path):  # 임베딩을 마친 데이터들은 이미 FAISS를 통해 저장되어 다시 가공될 필요 없음
        print("Vector Store Loading")
        vectorstore = FAISS.load_local(vectorstore_path, embeddings=embeddings, allow_dangerous_deserialization=True )  # Vector Store Load
    else:
        print("PDF -> Vector Store Creating Now..")
        documents = load_pdf_from_url(url)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)    # Splitter -> Chunk Size 조절 (하이퍼파라미터)
        split_docs = text_splitter.split_documents(documents)
        vectorstore = FAISS.from_documents(split_docs, embedding=embeddings)
        vectorstore.save_local(vectorstore_path)
        print(f"Vector Store has been stored in the'{vectorstore_path}'")

    return vectorstore

In [ ]:
from transformers import pipeline

def summarize_query_with_vectorstore(vectorstore, query):
    relevant_docs = vectorstore.similarity_search(query, k=5)     # 유사도 k=5 조절 가능 (하이퍼파라미터)
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")      # 오픈소스 LLM인 bart-large-cnn 사용
    combined_text = " ".join([doc.page_content for doc in relevant_docs])
    summary = summarizer(combined_text, max_length=150, min_length=40, do_sample=False)

    return summary[0]['summary_text']


In [ ]:
url = "https://arxiv.org/pdf/2005.11401.pdf"
vectorstore_path = "vectorstore_faiss"

# Vector Store 생성 또는 로드
vectorstore = process_pdf_and_create_vectorstore(url, vectorstore_path)

# 요약 생성
query = input("논문에서 알고 싶은 내용을 입력하세요: ")
summary = summarize_query_with_vectorstore(vectorstore, query)

print("요약 결과:")
print(summary)

Vector Store Loading
논문에서 알고 싶은 내용을 입력하세요: Who wrote this paper?
요약 결과:
RAG achieves an accuracy within 2.7% of this model, despite being supplied with only the claim and retrieving its own evidence. We also analyze whether documents retrieved by RAG correspond to documents annotated as gold.
